In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import nltk

# Lenguaje Claro - UPE
Lucas Pecina

09/01/21

---
### Objetivo
Comparar distintos textos (legales, judiciales, etc) para ver la complejidad que tiene ese texto en cuanto a palabras dificiles (poco frecuentes en el lenguaje cotidiano).

### Resumen
El lenguaje legal/judicial suele ser complicado de entender para la gente que no se dedica a temas relacionados debido a la complejidad de las palabras utilizadas y de su gramatica. El analisis de complejidad gramatical de los textos requiere la utilizacion de tecnicas muy sofisticadas y que no dan buenos resultados, por lo tanto, descartamos este tipo de estudio por el momento.

Comenzamos solo analizando las **palabras** de los textos y creamos un **indice de complejidad** que mide la cantidad y frecuencia de palabras "complejas", que son de poca frecuencia en el lenguaje cotidiano. 

No contaran para el calculo aquellas palabras que por su naturaleza tecnica juridica requiera de su uso obligatorio. Para dicha tarea, se elaborara a mano una lista de las palabras en cuestion.

### Fuentes de datos
- Textos a analizar: seran cargados como archivos de texto
 - Codigo Procesal Civil y Comercial actual http://servicios.infoleg.gob.ar/infolegInternet/anexos/15000-19999/16547/texact.htm esta en un archivo CPCC
- Bolsa de palabras de frecuencia del lenguaje español http://corpus.rae.es/lfrecuencias.html. Usando las 300mil palabras mas frecuentes del español: en datos/

### Pasos
1. Corpus de palabras frecuentes
 - Armar dataframe con frecuencias
 - Calcular complejidad de palabras
2. Cargar textos y preprocesarlo
 - Tokenizar las palabras
 - Limpiar errores
 - Remover stopwords


---
### Indice de complejidad del lenguaje
$$Complejidad_t = \frac{1}{P}\sum_p^P{Complejidad_p}$$

- t = texto a analizar
- p = lista de palabras en el texto t (pueden estar repetidas)
- P = cantidad de palabras en el texto t (todas)

$$Complejidad_p = f(FrecuenciaBolsa_p)$$
> **DETERMINAR EL INDICE DE COMPLEJIDAD DE LAS PALABRAS**. 
Ver la distribucion de frecuencias (despues de remover stopwords). Ver si hacerlo por frecuencia de palabras o ranking.
Puede ser el ranking o el inverso de las frecuencias.

$$Complejidad_p = \frac{1}{FrecuenciaBolsa_p}$$

o

$$Complejidad_p = RankingBolsa_p$$
 

Corpus de palabras frecuentes del español
---



In [147]:
# cargo datos
corpus = pd.read_excel('/Users/lucaspecina/Desktop/Data/Planificacion Estrategica/lenguaje-claro/upe-lenguaje_claro/datos/CREA_total.xlsx')
corpus = corpus.dropna()
corpus['Palabra'] = corpus.Palabra.astype('str').str.strip()
corpus = corpus[corpus.Frecuencia_norm>0.00]

# reemplazo caracteres mal cargados
reemplazar = {'·':'á','È':'é','Ò':'ñ','˙':'ú','¸':'ü','Ì':'í','Û':'ó'}

for k,v in reemplazar.items():
    corpus['Palabra'] = corpus.Palabra.str.replace(k,v)
corpus

,Orden,Palabra,Frecuencia,Frecuencia_norm
0,1,de,9999518,65545.55
1,2,la,6277560,41148.59
2,3,que,4681839,30688.85
3,4,el,4569652,29953.48
4,5,en,4234281,27755.16
...,...,...,...,...
423729,423730,zyscovich,2,0.01
423730,423731,zyx,2,0.01
423731,423732,zzz,2,0.01
423732,423733,zzzzzzzzzzz,2,0.01


In [148]:
corpus.Frecuencia_norm.sum()

995706.050000001

In [149]:
# calculo complejidad
corpus['complejidad'] = 1/corpus.Frecuencia_norm
corpus

,Orden,Palabra,Frecuencia,Frecuencia_norm,complejidad
0,1,de,9999518,65545.55,0.000015
1,2,la,6277560,41148.59,0.000024
2,3,que,4681839,30688.85,0.000033
3,4,el,4569652,29953.48,0.000033
4,5,en,4234281,27755.16,0.000036
...,...,...,...,...,...
423729,423730,zyscovich,2,0.01,100.000000
423730,423731,zyx,2,0.01,100.000000
423731,423732,zzz,2,0.01,100.000000
423732,423733,zzzzzzzzzzz,2,0.01,100.000000


Procesamiento de textos a analizar
---


In [150]:
# funcion para el preprocesamiento de los textos a cargar

def limpieza_textos(raw,corpus):
    tokens = nltk.word_tokenize(raw)
    tokens = [t.lower() for t in tokens if t.isalpha()]
    print(f'El texto original tiene {len(tokens)} palabras.')
    print(f'De esas, hay {len(np.unique(np.array(tokens)))} unicas')
    
    # hay muchos que finalizan en "are", 'ere'. Mencionare, juzgare, interpusiere, etc.
    # a esos los saco
    tokens = [t for t in tokens if ('are' not in t) & ('ere' not in t)]
    print(f'Despues de sacarle las palabras que terminan en ARE o ERE, quedan {len(tokens)} palabras.')
    print(f'Ahora hay {len(np.unique(np.array(tokens)))} unicas')
    
    # veo las palabras del texto que no estan en el corpus y las agrego al corpus para que cuenten
    no_estan = [t for t in np.unique(np.array(tokens)) if t not in corpus.Palabra.values]
    print(f'Hay {len(no_estan)} parabras unicas que no aparecen en el corpus. Las agrego con 100 de complejidad')
    
    # ahora quedaron solo las que son muy raras o las que estan sin tildes por los titulos
    '''Ahora hay que agregar todas estas palabras al df de corpus y agregarle
    el valor maximo = 100 '''

    no_estan_df = pd.DataFrame({'Palabra':no_estan, 'complejidad':[100.0]*len(no_estan)})
    print('no estan df')
    print(no_estan_df)
    # concat
    corpus_ag = pd.concat([corpus[['Palabra','complejidad']],no_estan_df]).reset_index()
    
    return(tokens,corpus_ag)    

In [ ]:
# funcion para el preprocesamiento de los textos a cargar

def limpieza_textos_simple(raw,corpus):
    '''Considera como errores/cosas extremas a las palabras que estan
    en el texto pero no en el corpus'''
    tokens = nltk.word_tokenize(raw)
    tokens = [t.lower() for t in tokens if t.isalpha()]
    print(f'El texto original tiene {len(tokens)} palabras.')
    print(f'De esas, hay {len(np.unique(np.array(tokens)))} unicas')

    return(tokens)

In [156]:
# funcion para calcular frecuencias de palabras en texto y calculo el indice_complejidad

def crear_indice_complejidad(tokens,corpus_ag,nombre_texto):
    '''1 crear df de palabras de nombre_texto y las frecuencias
       2 joinear y calcular'''

    tokens_freq = pd.DataFrame.from_dict(nltk.FreqDist(tokens), orient='index').reset_index()
    tokens_freq.columns = ['Palabra','freq']
    #tokens_freq['freq_relativa'] = tokens_freq.freq/tokens_freq.freq.sum()

    print('tokens_freq')
    print(tokens_freq)
    
    # joinear
    tokens_complejidad = corpus_ag.merge(tokens_freq, left_on='Palabra',right_on='Palabra')
    print('tokens_complejidad (join tokens_freq y corpus)')
    print(tokens_complejidad)
    
    # calculo indice_complejidad
    tokens_complejidad['indice_complejidad'] = (tokens_complejidad.complejidad * tokens_complejidad.freq)/tokens_complejidad.freq.sum()
    print('calculo indice_complejidad')
    print(tokens_complejidad)
    
    indice_complejidad = round(sum(tokens_complejidad.indice_complejidad),5)
    print(f'INDICE DE COMPLEJIDAD DE {nombre_texto}: {indice_complejidad}')
    
    return(tokens_complejidad, indice_complejidad)

CPCC
---

In [158]:
cpcc_f = open('/Users/lucaspecina/Desktop/Data/Planificacion Estrategica/lenguaje-claro/upe-lenguaje_claro/datos/CPCC.txt')
cpcc_raw = cpcc_f.read()
cpcc_tokens = limpieza_textos_simple(cpcc_raw,corpus)
cpcc_tokens, cpcc_indice = crear_indice_complejidad(cpcc_tokens,cpcc_corpus,'CPCC actual')
cpcc_tokens

El texto original tiene 65278 palabras.
De esas, hay 5637 unicas
tokens_freq
          Palabra  freq
0        caracter     8
1             art   939
2              la  3509
3     competencia    29
4       atribuida     1
...           ...   ...
5632  controlarlo     1
5633    protestas     1
5634     escritas     1
5635     quisiera     1
5636      faculta     1

[5637 rows x 2 columns]
tokens_complejidad (join tokens_freq y corpus)
      index       Palabra  complejidad  freq
0         0            de     0.000015  4493
1         1            la     0.000024  3509
2         2           que     0.000033  1682
3         3            el     0.000033  2731
4         4            en     0.000036  1918
...     ...           ...          ...   ...
5376    100  substanciará   100.000000     1
5377    102    sumarisimo   100.000000     3
5378    105   sustanciará   100.000000    17
5379    106  sustanciarán   100.000000     7
5380    107     tercerias   100.000000     1

[5381 rows x 4 columns

,index,Palabra,complejidad,freq,indice_complejidad
0,0,de,0.000015,4493,1.057868e-06
1,1,la,0.000024,3509,1.316033e-06
2,2,que,0.000033,1682,8.458313e-07
3,3,el,0.000033,2731,1.407061e-06
4,4,en,0.000036,1918,1.066457e-06
...,...,...,...,...,...
5376,100,substanciará,100.000000,1,1.543258e-03
5377,102,sumarisimo,100.000000,3,4.629773e-03
5378,105,sustanciará,100.000000,17,2.623538e-02
5379,106,sustanciarán,100.000000,7,1.080280e-02


CPP
---

In [160]:
cpp_f = open('/Users/lucaspecina/Desktop/Data/Planificacion Estrategica/lenguaje-claro/upe-lenguaje_claro/datos/CPP.txt')
cpp_raw = cpp_f.read()

cpp_tokens = limpieza_textos_simple(cpp_raw,corpus)
cpp_tokens, cpp_indice = crear_indice_complejidad(cpp_tokens,cpp_corpus,'CPP actual')

El texto original tiene 44432 palabras.
De esas, hay 4445 unicas
tokens_freq
           Palabra  freq
0           codigo    11
1         procesal    29
2            penal   101
3              ley   163
4              ver     2
...            ...   ...
4440  permanecerán     1
4441       entrará     1
4442     efectuada     1
4443       reforma     1
4444       órganos     1

[4445 rows x 2 columns]
tokens_complejidad (join tokens_freq y corpus)
      index           Palabra  complejidad  freq
0         0                de     0.000015  2930
1         1                la     0.000024  2044
2         2               que     0.000033  1106
3         3                el     0.000033  1804
4         4                en     0.000036  1260
...     ...               ...          ...   ...
4393     36    sobrerraspados   100.000000     1
4394     37        sobreseerá   100.000000     3
4395     38      substanciará   100.000000     1
4396     39     substanciarán   100.000000     1
4397     40 

CPCC2
---

In [161]:
cpcc_f = open('/Users/lucaspecina/Desktop/Data/Planificacion Estrategica/lenguaje-claro/upe-lenguaje_claro/datos/CPCC2.txt')
cpcc_raw = cpcc_f.read()
cpcc_tokens = limpieza_textos_simple(cpcc_raw,corpus)
cpcc_tokens, cpcc_indice = crear_indice_complejidad(cpcc_tokens,cpcc_corpus,'CPCC actual')
cpcc_tokens

El texto original tiene 62785 palabras.
De esas, hay 5627 unicas
tokens_freq
            Palabra  freq
0             libro    23
1                 i    32
2     disposiciones    45
3         generales    16
4            título    70
...             ...   ...
5622       cúmplase     1
5623    comuníquese     1
5624     publíquese     1
5625           dese     1
5626      archívese     1

[5627 rows x 2 columns]
tokens_complejidad (join tokens_freq y corpus)
      index        Palabra  complejidad  freq
0         0             de     0.000015  3926
1         1             la     0.000024  3018
2         2            que     0.000033  1678
3         3             el     0.000033  2600
4         4             en     0.000036  1861
...     ...            ...          ...   ...
5463    104     suscitasen   100.000000     1
5464    105    sustanciará   100.000000    15
5465    106   sustanciarán   100.000000     6
5466    107      tercerias   100.000000     1
5467    108  testimoniarse   100.

,index,Palabra,complejidad,freq,indice_complejidad
0,0,de,0.000015,3926,9.584945e-07
1,1,la,0.000024,3018,1.173672e-06
2,2,que,0.000033,1678,8.749714e-07
3,3,el,0.000033,2600,1.389020e-06
4,4,en,0.000036,1861,1.072964e-06
...,...,...,...,...,...
5463,104,suscitasen,100.000000,1,1.600230e-03
5464,105,sustanciará,100.000000,15,2.400346e-02
5465,106,sustanciarán,100.000000,6,9.601383e-03
5466,107,tercerias,100.000000,1,1.600230e-03


CPP
---

In [84]:
f = open('/Users/lucaspecina/Desktop/Data/Planificacion Estrategica/lenguaje-claro/upe-lenguaje_claro/datos/CPP.txt')
raw = f.read()
CPP_tokens = nltk.word_tokenize(raw)

In [85]:
CPP_tokens = [t.lower() for t in CPP_tokens if t.isalpha()]

In [86]:
CPP_tokens = [t for t in CPP_tokens if ('are' not in t) & ('ere' not in t)]

In [87]:
# HAGO LA PRIMERA PRUEBA
'''
1 crear df de palabras en CPCC y las frecuencias
2 joinear y calcular
'''

CPP_actual_freq = pd.DataFrame.from_dict(nltk.FreqDist(CPP_tokens), orient='index').reset_index()
CPP_actual_freq.columns = ['Palabra','freq']

In [88]:
# joinear

CPP_actual_complejidad = corpus_ag.merge(CPP_actual_freq, left_on='Palabra',right_on='Palabra')

In [89]:
# calculo complej_ponderado

CPP_actual_complejidad['freq_relativa'] = CPP_actual_complejidad.freq/CPP_actual_complejidad.freq.sum()
CPP_actual_complejidad['complejidad_ponderado'] = CPP_actual_complejidad.complejidad * CPP_actual_complejidad.freq_relativa
CPP_actual_complejidad

,index,Palabra,complejidad,freq,freq_relativa,complejidad_ponderado
0,0,de,0.000015,2930,0.067384,1.028052e-06
1,1,la,0.000024,2044,0.047008,1.142395e-06
2,2,que,0.000033,1106,0.025436,8.288291e-07
3,3,el,0.000033,1804,0.041488,1.385096e-06
4,4,en,0.000036,1260,0.028978,1.044040e-06
...,...,...,...,...,...,...
4197,164,obstará,100.000000,2,0.000046,4.599604e-03
4198,166,oponerlas,100.000000,1,0.000023,2.299802e-03
4199,222,substanciación,100.000000,1,0.000023,2.299802e-03
4200,223,substanciará,100.000000,1,0.000023,2.299802e-03


In [90]:
print(f'INDICE DE COMPLEJIDAD DE CPP ACTUAL: {round(sum(CPP_actual_complejidad.complejidad_ponderado),5)}')

INDICE DE COMPLEJIDAD DE CPP ACTUAL: 0.33905
